In [25]:
# bob.refresh {"cron": "10,35 8,11,19,21 * * *", "timeout": 40000}
# bob.static {"token": "4622c4cc3822f6c1164cadbd53ed0d1230fb0a562d7c6bc824116cd82a78", "path": "/home/ftp/ftp/dev/wsr-public/data_output/France_HOSP_DEP_NAME.html", "filename": "../data_output/France_HOSP_DEP_NAME.html"}
# bob.static {"token": "24e8978d3f09f857f81ed59d08c2b01a98e7eb0a95b85d985158b449bf0a", "path": "/home/ftp/ftp/dev/wsr-public/data_output/France_REA_DEP_NAME.html", "filename": "../data_output/France_REA_DEP_NAME.html"} 
# bob.static {"token": "83596ba6c97aa043e1994c93e4fc49354cd31842c2f83d66da0ce9dc314f", "path": "/home/ftp/ftp/dev/wsr-public/data_output/France_DC_DEP_NAME.html", "filename": "../data_output/France_DC_DEP_NAME.html"} 
# bob.static {"token": "09e1f0c28332e7b570cb13fe66def2187a457697a9a54921c96cfb2f76fd", "path": "/home/ftp/ftp/dev/wsr-public/data_output/France_RAD_DEP_NAME.html", "filename": "../data_output/France_RAD_DEP_NAME.html"} 

# bob.static {"token": "113a1888cfa76275418845a0a21fbed1e2f766bf12a0a7b9b72c20de1aa5", "path": "/home/ftp/ftp/dev/wsr-public/data_output/France_HOSP_REG_NAME.html", "filename": "../data_output/France_HOSP_REG_NAME.html"}
# bob.static {"token": "1bf05f448efec2591a9b59b2a8683b24eb99eaf3314f7a6d87937d05f7f4", "path": "/home/ftp/ftp/dev/wsr-public/data_output/France_REA_REG_NAME.html", "filename": "../data_output/France_REA_REG_NAME.html"}
# bob.static {"token": "a1e16c779ba48901c40ad7daf39dceea2dda8752b6de257b58108a97b9eb", "path": "/home/ftp/ftp/dev/wsr-public/data_output/France_DC_REG_NAME.html", "filename": "../data_output/France_DC_REG_NAME.html"} 
# bob.static {"token": "5fd5e5d8525afa27210b5c64420d475a531c3b1eb3d6bbd9fed979f3081d", "path": "/home/ftp/ftp/dev/wsr-public/data_output/France_RAD_REG_NAME.html", "filename": "../data_output/France_RAD_REG_NAME.html"} 

In [26]:
%run __init__.ipynb

Successfully connected to MongoDB


In [41]:
#--- Check if the data are already updated
filePath = f'{output_folder}107_FRANCE.csv'
if path.exists(filePath):
    df_check = pd.read_csv(filePath, sep=';',low_memory=False)
    date_check = df_check[df_check['SCENARIO'] == yesterday.strftime('%d/%m/%Y')]
    if len(date_check) > 0:
        raise Exception(f'Data already updated ! DateTime : {now}')


Exception: Data already updated ! DateTime : 30/04/2020 13:05:08

In [28]:
#Source 107
df = pd.read_csv(output_folder + '107.csv',sep=";").drop('Unnamed: 0',axis=1)
df = df[(df['SCENARIO'] == yesterday.strftime('%d/%m/%Y'))].reset_index(drop=True)
if len(df) == 0:
    sys.exit(f'Data not updated ! DateTime : {now}')
else:
    df.to_csv(output_folder + '107_FRANCE.csv',sep=";")

ref_france['DEP_CODE'] = ref_france['DEP_CODE'].astype(str)
ref_france['DEP_NAME'] = ref_france['DEP_NAME'].astype(str)
ref_france['DEP_FULL'] = ref_france['DEP_CODE'] + ' - ' + ref_france['DEP_NAME']
ref_france

df = pd.merge(df,ref_france, left_on='CHILD', right_on='DEP_FULL',how='left').rename(index=str, columns={"DEP_NAME":"LABEL"}).drop(['DEP_CODE','DEP_FULL','REG_NAME'], axis=1)
df.loc[df['PARENT'] == 'root', 'LABEL'] = df['CHILD']
df = df.drop(['UNIT_VARP','PRECISION_VALUE','PRECISION_VAR','PRECISION_VARP','LAST_UPDATE'],axis=1)
df.head(5)

,ENTITY,SCENARIO,DATE_ORDER,CHILD,PARENT,UPPER_FILTER_M,ORDER,VALUE,VAR,VARP,LABEL
0,France,29/04/2020,20200429,Île-de-France,root,En réanimation,2.0,1680,-121,-6.718490,Île-de-France
1,France,29/04/2020,20200429,Île-de-France,root,Hospitalisations,1.0,11085,-264,-2.326196,Île-de-France
2,France,29/04/2020,20200429,31 - Haute-Garonne,Occitanie,Hospitalisations,1.0,131,-10,-7.092199,Haute-Garonne
3,France,29/04/2020,20200429,32 - Gers,Occitanie,Hospitalisations,1.0,35,-5,-12.500000,Gers
4,France,29/04/2020,20200429,33 - Gironde,Nouvelle-Aquitaine,Hospitalisations,1.0,234,-20,-7.874016,Gironde


In [29]:
def analysis(df,variable):
    Min = df[variable].min()
    Max = df[variable].max()
    Avr = round(df[variable].mean(),0)
    Med = round(np.median(df[variable]),0)
    AvrMax = round((Avr / Max),2)
    MedMax = round((Med / Max),2)
    to_print = f'{variable} : Min: {Min}, Max: {Max}, Average: {Avr}, AvrMax: {AvrMax}, Med: {Med}, MedMax: {MedMax}'
    return print(to_print)

In [30]:
#Var France
map_center = {"lat": 46.7667, "lon": 2.45}
def create_map(df,geo,dfKey,geoKey,scale,colors,map_center,label,file_html):
    fig = px.choropleth_mapbox(df, 
                               geojson=geo, 
                               color="VALUE",
                               locations=dfKey, 
                               featureidkey=geoKey,
                               range_color=scale,
                               color_continuous_scale=colors,
                               mapbox_style="carto-positron", 
                               zoom=5, 
                               center=map_center,
                               hover_data=["VALUE","VAR"],
                               labels=label,
                              )
    fig.update_layout(margin={"r":0,"l":0,},autosize=True)
    fig.write_html(f"{output_folder}{file_html}")
    updateChartCss(f"{output_folder}{file_html}", f"{input_folder}map.css")
#     return fig

In [31]:
#Hospitalisation - Département
kpi = 'Hospitalisations'
file_html = "France_HOSP_DEP_NAME.html"

df_map = df.copy()[(df['UPPER_FILTER_M'] == kpi) & (df['PARENT'] != 'root')]
analysis(df_map,'VALUE')

scale = (0,3000)
colors = [
    [0.0, "#FFFFFF"],
    [0.05,"#B3D7D2"],
    [0.10, "#81B1BD"],
    [0.55, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "LABEL"
geoKey="properties.nom"
to_rename = {'LABEL':'Département','VALUE': kpi, 'VAR': 'Variation en nb.'}
create_map(df_map,france_dep,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)
# bob.makeStaticGettable(output_folder + file_html,"dev")

VALUE : Min: 1, Max: 2348, Average: 265.0, AvrMax: 0.11, Med: 113.0, MedMax: 0.05


In [32]:
#En réanimation - Département
kpi = 'En réanimation'
file_html = "France_REA_DEP_NAME.html"

df_map = df.copy()[(df['UPPER_FILTER_M'] == kpi) & (df['PARENT'] != 'root')]
analysis(df_map,'VALUE')

scale = (0,800)
colors = [
    [0.0, "#FFFFFF"],
    [0.025,"#B3D7D2"],
    [0.10, "#81B1BD"],
    [0.55, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "LABEL"
geoKey="properties.nom"
to_rename = {'LABEL':'Département','VALUE': kpi, 'VAR': 'Variation en nb.'}
create_map(df_map,france_dep,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)
# bob.makeStaticGettable(output_folder + file_html,"dev")

VALUE : Min: 0, Max: 468, Average: 41.0, AvrMax: 0.09, Med: 14.0, MedMax: 0.03


In [33]:
#"Décès à l’hôpital" - Département
kpi = "Décès à l’hôpital"
file_html = "France_DC_DEP_NAME.html"

df_map = df.copy()[(df['UPPER_FILTER_M'] == kpi) & (df['PARENT'] != 'root')]
analysis(df_map,'VALUE')

scale = (0,1500)
colors = [
    [0.0, "#FFFFFF"],
    [0.05,"#B3D7D2"],
    [0.10, "#81B1BD"],
    [0.55, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "LABEL"
geoKey="properties.nom"
to_rename = {'LABEL':'Département','VALUE': kpi, 'VAR': 'Variation en nb.'}
create_map(df_map,france_dep,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)
# bob.makeStaticGettable(output_folder + file_html,"dev")

VALUE : Min: 0, Max: 1446, Average: 149.0, AvrMax: 0.1, Med: 58.0, MedMax: 0.04


In [34]:
#'Retours à domicile' - Département
kpi = 'Retours à domicile'
file_html = "France_RAD_DEP_NAME.html"

df_map = df.copy()[(df['UPPER_FILTER_M'] == kpi) & (df['PARENT'] != 'root')]
analysis(df_map,'VALUE')

scale = (0,3000)
colors = [
    [0.0, "#FFFFFF"],
    [0.05,"#B3D7D2"],
    [0.10, "#81B1BD"],
    [0.55, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "LABEL"
geoKey="properties.nom"
to_rename = {'LABEL':'Département','VALUE': kpi, 'VAR': 'Variation en nb.'}
create_map(df_map,france_dep,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)
# bob.makeStaticGettable(output_folder + file_html,"dev")

VALUE : Min: 15, Max: 3972, Average: 477.0, AvrMax: 0.12, Med: 170.0, MedMax: 0.04


#ANALYSIS BY REGION

In [35]:
#Hospitalisation - Région
kpi = 'Hospitalisations'
file_html = "France_HOSP_REG_NAME.html"

df_map = df.copy()[(df['UPPER_FILTER_M'] == kpi) & (df['PARENT'] == 'root') & (df['LABEL'] != 'France')]
analysis(df_map,'VALUE')

scale = (100,13000)
colors = [
    [0.0, "#FFFFFF"],
    [0.075,"#B3D7D2"],
    [0.15, "#81B1BD"],
    [0.4, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "LABEL"
geoKey="properties.nom"
to_rename = {'LABEL':'Région','VALUE': kpi, 'VAR': 'Variation en nb.'}
create_map(df_map,france_reg,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)
# bob.makeStaticGettable(output_folder + file_html,"dev")

VALUE : Min: 1, Max: 11085, Average: 1485.0, AvrMax: 0.13, Med: 656.0, MedMax: 0.06


In [36]:
#En réanimation - Région
kpi = 'En réanimation'
file_html = "France_REA_REG_NAME.html"

df_map = df.copy()[(df['UPPER_FILTER_M'] == kpi) & (df['PARENT'] == 'root') & (df['LABEL'] != 'France')]
analysis(df_map,'VALUE')

scale = (50,2500)
colors = [
    [0.0, "#FFFFFF"],
    [0.075,"#B3D7D2"],
    [0.15, "#81B1BD"],
    [0.55, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "LABEL"
geoKey="properties.nom"
to_rename = {'LABEL':'Région','VALUE': kpi, 'VAR': 'Variation en nb.'}
create_map(df_map,france_reg,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)
# bob.makeStaticGettable(output_folder + file_html,"dev")

VALUE : Min: 0, Max: 1680, Average: 229.0, AvrMax: 0.14, Med: 108.0, MedMax: 0.06


In [37]:
#"Décès à l’hôpital" - Région
kpi = "Décès à l’hôpital"
file_html = "France_DC_REG_NAME.html"

df_map = df.copy()[(df['UPPER_FILTER_M'] == kpi) & (df['PARENT'] == 'root') & (df['LABEL'] != 'France')]
analysis(df_map,'VALUE')

scale = (100,5500)
colors = [
    [0.0, "#FFFFFF"],
    [0.075,"#B3D7D2"],
    [0.15, "#81B1BD"],
    [0.4, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "LABEL"
geoKey="properties.nom"
to_rename = {'LABEL':'Région','VALUE': kpi, 'VAR': 'Variation en nb.'}
create_map(df_map,france_reg,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)
# bob.makeStaticGettable(output_folder + file_html,"dev")

VALUE : Min: 0, Max: 5938, Average: 836.0, AvrMax: 0.14, Med: 340.0, MedMax: 0.06


In [38]:
#Retours à domicile - Région
kpi = 'Retours à domicile'
file_html = "France_RAD_REG_NAME.html"

df_map = df.copy()[(df['UPPER_FILTER_M'] == kpi) & (df['PARENT'] == 'root') & (df['LABEL'] != 'France')]
analysis(df_map,'VALUE')

scale = (200,15000)
colors = [
    [0.0, "#FFFFFF"],
    [0.075,"#B3D7D2"],
    [0.15, "#81B1BD"],
    [0.4, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "LABEL"
geoKey="properties.nom"
to_rename = {'LABEL':'Région','VALUE': kpi, 'VAR': 'Variation en nb.'}
create_map(df_map,france_reg,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)
# bob.makeStaticGettable(output_folder + file_html,"dev")

VALUE : Min: 35, Max: 17140, Average: 2679.0, AvrMax: 0.16, Med: 1198.0, MedMax: 0.07


In [39]:
#html
#Par département
hosp_dep = [{'URL' : 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/4622c4cc3822f6c1164cadbd53ed0d1230fb0a562d7c6bc824116cd82a78', 'KPI' : 'Hospitalisations', 'UPPER_FILTER' : 'Par département'}]
rea_dep = [{'URL' : 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/24e8978d3f09f857f81ed59d08c2b01a98e7eb0a95b85d985158b449bf0a', 'KPI' : 'En réanimation', 'UPPER_FILTER' : 'Par département'}]
dc_dep = [{'URL' : 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/09e1f0c28332e7b570cb13fe66def2187a457697a9a54921c96cfb2f76fd', 'KPI' : 'Retours à domicile', 'UPPER_FILTER' : 'Par département'}]
rad_dep = [{'URL' :'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/83596ba6c97aa043e1994c93e4fc49354cd31842c2f83d66da0ce9dc314f', 'KPI' : 'Décès à l’hôpital', 'UPPER_FILTER' : 'Par département'}]
          
#Par région
hosp_reg = [{'URL' : 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/113a1888cfa76275418845a0a21fbed1e2f766bf12a0a7b9b72c20de1aa5', 'KPI' : 'Hospitalisations', 'UPPER_FILTER' : 'Par région'}]
rea_reg = [{'URL' : 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/1bf05f448efec2591a9b59b2a8683b24eb99eaf3314f7a6d87937d05f7f4', 'KPI' : 'En réanimation', 'UPPER_FILTER' : 'Par région'}]
dc_reg = [{'URL' : 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/5fd5e5d8525afa27210b5c64420d475a531c3b1eb3d6bbd9fed979f3081d', 'KPI' : 'Retours à domicile', 'UPPER_FILTER' : 'Par région'}]
rad_reg = [{'URL' : 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/a1e16c779ba48901c40ad7daf39dceea2dda8752b6de257b58108a97b9eb', 'KPI' : 'Décès à l’hôpital', 'UPPER_FILTER' : 'Par région'}]
        
list_kpis = hosp_dep + rea_dep + dc_dep + rad_dep + hosp_reg+ rea_reg + dc_reg + rad_reg
domain = pd.DataFrame(list_kpis)
domain.loc[:,'VALUE'] = """<iframe src='"""+ domain['URL'] +"""' frameborder='0' style='overflow:hidden;height:85vh;width:100%' height='100%' width='100%'></iframe> <style>.modebar{display: none!important;}</style>"""
if use_mongo:
    bob.mongo.save_df(domain,'110',db_app,True)
domain

Dataframe 110 successfully save in database app-wsr-dev in MongoDB. Time: --- 0.016390323638916016 secnds ---


,URL,KPI,UPPER_FILTER,VALUE
0,https://api.dev.cashstory.com/api/v1/notebook/...,Hospitalisations,Par département,<iframe src='https://api.dev.cashstory.com/api...
1,https://api.dev.cashstory.com/api/v1/notebook/...,En réanimation,Par département,<iframe src='https://api.dev.cashstory.com/api...
2,https://api.dev.cashstory.com/api/v1/notebook/...,Retours à domicile,Par département,<iframe src='https://api.dev.cashstory.com/api...
3,https://api.dev.cashstory.com/api/v1/notebook/...,Décès à l’hôpital,Par département,<iframe src='https://api.dev.cashstory.com/api...
4,https://api.dev.cashstory.com/api/v1/notebook/...,Hospitalisations,Par région,<iframe src='https://api.dev.cashstory.com/api...
5,https://api.dev.cashstory.com/api/v1/notebook/...,En réanimation,Par région,<iframe src='https://api.dev.cashstory.com/api...
6,https://api.dev.cashstory.com/api/v1/notebook/...,Retours à domicile,Par région,<iframe src='https://api.dev.cashstory.com/api...
7,https://api.dev.cashstory.com/api/v1/notebook/...,Décès à l’hôpital,Par région,<iframe src='https://api.dev.cashstory.com/api...
